# Scraping Steam

Este cuaderno Jupyter contiene el codigo fuente encargado de la extraccion automatica de datos desde la plataforma **Steam**. El objetivo es alimentar un modelo de IA con informacion actualizada sobre:

1.  **Hardware mas utilizado:** Componentes reales que usan los jugadores (GPUs, CPUs, RAM, SO, etc.).

2.  **Requisitos de videojuegos:** Especificaciones minimas y recomendadas de los titulos mas vendidos.

El proceso genera dos tipos de salidas estructuradas (`CSV` y `JSON`) para su posterior procesamiento en la fase de entrenamiento del modelo.

### 👨‍💻 Autores del proyecto

* [Alejandro Barrionuevo Rosado](https://github.com/Alejandro-BR)
* [Alvaro López Guerrero](https://github.com/Alvalogue72)
* [Andrei Munteanu Popa](https://github.com/andu8705)

Máster de FP en Inteligencia Artifical y Big Data - CPIFP Alan Turing - `Curso 2025/2026`

### 1. Configuracion e importacion de librerias

En esta primera celda importamos las herramientas necesarias:

* `requests`: Para realizar las peticiones HTTP a los servidores de Steam.

* `BeautifulSoup`: Para navegar y extraer informacion del codigo HTML.

* `pandas`: Para estructurar los datos en tablas y exportarlos facilmente.

* `re`: Para el uso de expresiones regulares, cruciales para limpiar el texto sucio de los requisitos.

Tambien definimos `GAME_LIMIT`, que determina cuantos juegos analizara el script.

In [1]:
import os
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Define el volumen de datos a recolectar:
GAME_LIMIT = 6000

### 2. Gestion del sistema de archivos

Definimos una funcion auxiliar para mantener el proyecto ordenado. Esta funcion crea automaticamente la estructura de carpetas necesaria (`csv_data` y `json_data`) antes de empezar, asegurando que no haya errores de "ruta no encontrada" al guardar los archivos.

In [2]:
def create_structure():
    dirs = ['csv_data', 'json_data', 'csv_data/ranked_hardware', 'json_data/ranked_hardware']
    for d in dirs:
        os.makedirs(d, exist_ok=True)

### 3. Extraccion de la encuesta de hardware

Esta seccion aborda el *scraping* de la pagina de estadisticas de Steam.

La estructura de esta web es compleja, mezclando encabezados con listas de detalles ocultas. La funcion `get_steam_hardware_survey` implementa la siguiente logica:

1.  Identifica las filas principales (categorias).

2.  Busca los contenedores de detalles hijos (`stats_row_details`).

3.  Asocia correctamente cada componente con su porcentaje de uso real, ignorando filas vacias o datos de cambio mensual irrelevantes.

In [3]:
# Crea las carpetas y subcarpetas necesarias para almacenar los datos.
def get_steam_hardware_survey():
    url = "https://store.steampowered.com/hwsurvey"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        
        data = []
        
        # Buscamos todos los "divs" que sean fila o detalle
        all_divs = soup.find_all('div', class_=lambda x: x and ('stats_row' in x or 'stats_row_details' in x))
        
        current_category = "General"
        
        for div in all_divs:
            classes = div.get('class', [])
            
            # 1. DETECTAR CATEGORIA
            if 'stats_row' in classes and 'stats_row_details' not in classes:
                col_left = div.select_one('.stats_col_left')
                if col_left:
                    text = col_left.get_text(strip=True)
                    if text:
                        current_category = text
            
            # 2. EXTRAER DATOS
            if 'stats_row_details' in classes:
                mids = div.select('.stats_col_mid')   # Nombres de componentes
                rights = div.select('.stats_col_right') # Porcentajes
                
                # Logica para emparejar columnas (evitar columnas de "cambio %")
                pcts = []
                if len(rights) >= 2 * len(mids) and len(mids) > 0:
                    pcts = rights[0::2] # Tomar solo columnas pares
                elif len(rights) == len(mids):
                    pcts = rights
                else:
                    pcts = rights[:len(mids)]
                
                for name_div, pct_div in zip(mids, pcts):
                    item_name = name_div.get_text(strip=True)
                    item_pct = pct_div.get_text(strip=True)
                    
                    if item_name and item_pct:
                        data.append({
                            'category': current_category,
                            'component': item_name,
                            'usage_percentage': item_pct
                        })
                        
        return pd.DataFrame(data)
    except Exception as e:
        print(f"Error scraping Hardware Survey: {e}")
        return pd.DataFrame()

### 4. Clasificacion y ranking de hardware

Una vez obtenidos los datos brutos del hardware, necesitamos organizarlos por relevancia. La funcion `save_ranked_hardware` procesa el DataFrame completo y genera tres tipos de archivos:

1.  **Top 1 (Most Used):** El componente mas popular de cada categoria.

2.  **Top 2:** El segundo mas popular.

3.  **Top 3:** El tercero mas popular.

Esto permite al modelo de IA entender la "gama media" y "gama alta" actual del mercado.

In [4]:
# Funcion auxiliar para guardar en CSV y JSON.
def save_datasets(df, filename_base):
    if df.empty: return
    
    # Reordenar columnas solo si es un dataset de juegos para poner app_id primero
    if 'app_id' in df.columns:
        cols = ['app_id', 'game_name'] + [c for c in df.columns if c not in ['app_id', 'game_name']]
        df = df[cols]
    
    csv_path = os.path.join('csv_data', f'{filename_base}.csv')
    json_path = os.path.join('json_data', f'{filename_base}.json')
    
    df.to_csv(csv_path, index=False, encoding='utf-8')
    df.to_json(json_path, orient='records', indent=4, force_ascii=False)

# Genera los ficheros de ranking por categoria.
def save_ranked_hardware(df):
    if df.empty: return

    # Limpieza
    df = df.copy()
    df['numeric_pct'] = df['usage_percentage'].str.extract(r'(\d+\.?\d*)').astype(float)
    
    # Ordenar
    df_sorted = df.sort_values(by=['category', 'numeric_pct'], ascending=[True, False])
    
    ranks_info = [
        (0, 'top1_hw'),
        (1, 'top2_hw'),
        (2, 'top3_hw')
    ]
    
    for idx, filename in ranks_info:
        # Obtener el n-ésimo elemento de cada grupo
        ranked_df = df_sorted.groupby('category').nth(idx).reset_index()
        
        if 'numeric_pct' in ranked_df.columns:
            ranked_df = ranked_df.drop(columns=['numeric_pct'])
            
        if not ranked_df.empty:
            # Guardamos en la subcarpeta 'ranked_hardware'
            filename_full = f"ranked_hardware/{filename}"
            save_datasets(ranked_df, filename_full)

### 5. Obtencion de identificadores de juegos

Steam no permite descargar una lista de un gran volumen juegos en una sola peticion (limita los resultados a 50 o 100 por consulta). Para superar esta barrera, se ha implementado un algoritmo de **paginacion automatica**.

La función `get_top_game_ids`:

1.  Funciona como un bucle que solicita juegos en **bloques de 50 en 50** (`batch_size = 50`).

2.  Utiliza un parametro `start` que se incrementa en cada iteracion para avanzar por la lista.

3.  Acumula los IDs encontrados hasta llegar al `GAME_LIMIT` deseado.

4.  Incluye una pausa de seguridad de 1.5 segundos entre bloques para evitar que Steam bloquee nuestra conexion por exceso de peticiones.

In [5]:
def get_top_game_ids(limit=100):
    all_ids = []
    batch_size = 50 # Pedimos bloques de 50 para no saturar ni ser bloqueados
    base_url = "https://store.steampowered.com/search/results/"
    
    # Bucle que avanza de 50 en 50 
    for start in range(0, limit, batch_size):
        params = {
            'query': '',
            'start': start,
            'count': batch_size,
            'dynamic_data': '',
            'sort_by': '_ASC',
            'snr': '1_7_7_7000_7',
            'filter': 'topsellers',
            'infinite': 1,
            'l': 'english'
        }
        
        try:
            response = requests.get(base_url, params=params)
            
            if response.status_code != 200:
                print(f"Error en bloque {start}: Status {response.status_code}")
                continue
                
            json_resp = response.json()
            html = json_resp.get('results_html', '')
            
            if not html:
                print("No se recibieron más datos de Steam.")
                break
                
            soup = BeautifulSoup(html, 'lxml')
            links = soup.select('a.search_result_row')
            
            batch_ids = [link.get('data-ds-appid') for link in links if link.get('data-ds-appid')]
            
            # Evitar duplicados si Steam repite resultados
            new_ids = [bid for bid in batch_ids if bid not in all_ids]
            all_ids.extend(new_ids)
            
            print(f" Bloque {start}-{start+batch_size} obtenido. Total: {len(all_ids)}")
            
            if len(all_ids) >= limit:
                break
                
            time.sleep(1.5) # Pausa importante para evitar bloqueo de IP
            
        except Exception as e:
            print(f"Error obteniendo IDs: {e}")
            break
            
    return all_ids[:limit]

### 6. Procesamiento inteligente de texto (Regex y limpieza)

Esta es una de las partes más criticas. Los requisitos en Steam son texto libre (HTML sucio). Aqui utilizamos funciones avanzadas para:

1.  **Limpiar HTML:** Convertir etiquetas `<br>` en saltos de linea.

2.  **Eliminar ruido:** Quitamos frases repetitivas como *"Requires a 64-bit processor and operating system"* que confunden al analisis.

3.  **Parsing con Regex:** Buscamos patrones especificos (`OS:`, `Processor:`, `Memory:`) para separar el texto en columnas utiles.

4.  **Validacion de procesador:** Si el campo "Processor" contiene texto basura residual ("and operating system"), se anula automaticamente para mantener la calidad del dato.

In [6]:
def clean_html_text(html_content):
    if not html_content: return ""
    text = str(html_content).replace('<br>', '\n').replace('<br/>', '\n')
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text(separator='\n', strip=True)

def parse_requirements(raw_text):
    if not raw_text: return None
    
    # 1. Limpieza General y de BOILERPLATE

        # Quitamos etiquetas comunes y la frase problematica de 64-bits
    raw_text = re.sub(r'^(Minimum|Recommended|Mínimo|Recomendado):?', '', raw_text, flags=re.IGNORECASE).strip()
    raw_text = re.sub(r'Requires a 64-bit processor and operating system', '', raw_text, flags=re.IGNORECASE).strip()

        # Patrones Regex para extraer componentes
    patterns = {
        'os': r'(?:OS|Sistema Operativo|SO)\s*:?\s*(.*?)(?=(?:Processor|Procesador|Memory|Memoria|Graphics|Gráficos|DirectX|Storage|Almacenamiento|Sound Card|$))',
        'processor': r'(?:Processor|Procesador)\s*:?\s*(.*?)(?=(?:Memory|Memoria|Graphics|Gráficos|DirectX|Storage|Almacenamiento|Sound Card|OS|Sistema|$))',
        'memory': r'(?:Memory|Memoria)\s*:?\s*(.*?)(?=(?:Graphics|Gráficos|DirectX|Storage|Almacenamiento|Sound Card|OS|Sistema|Processor|$))',
        'graphics': r'(?:Graphics|Gráficos|Video Card|Vídeo)\s*:?\s*(.*?)(?=(?:DirectX|Storage|Almacenamiento|Sound Card|OS|Sistema|Processor|Memory|$))',
        'directx': r'(?:DirectX)\s*:?\s*(.*?)(?=(?:Storage|Almacenamiento|Sound Card|OS|Sistema|Processor|Memory|Graphics|$))',
        'storage': r'(?:Storage|Almacenamiento|Hard Drive)\s*:?\s*(.*?)(?=(?:Sound Card|Additional Notes|Notas|OS|Sistema|$))'
    }

    parsed_data = {}
    found_any = False
    
    for key, pattern in patterns.items():
        match = re.search(pattern, raw_text, re.IGNORECASE | re.DOTALL)
        if match:
            value = match.group(1).strip().strip(',').strip(';')
            parsed_data[key] = value
            if value: found_any = True
        else:
            parsed_data[key] = None
            
        # Si no se encontro ningun campo util, descartamos la fila
    if not found_any: return None

    # 2. FILTRO DE CALIDAD PARA PROCESADOR

        # Si detectamos texto basura en la columna processor, lo marcamos como nulo
    proc_val = parsed_data.get('processor')
    if proc_val and 'and operating system' in proc_val.lower():
        parsed_data['processor'] = None

    return parsed_data

### 7. Extraccion masiva de requisitos

Esta funcion itera sobre la lista de IDs de juegos y consulta la **API oficial de Steam** (`appdetails`).

* Fuerza el idioma ingles (`l=english`) para maximizar la compatibilidad del Regex.

* Separa requisitos minimos y recomendados.

* Incluye una pausa (`time.sleep`) para respetar los limites de la API y evitar bloqueos de IP.

In [7]:
def get_game_requirements(app_ids):
    min_data = []
    rec_data = []
    base_url = "https://store.steampowered.com/api/appdetails"
    
    for idx, app_id in enumerate(app_ids):
        # Log de progreso cada 10 juegos
        if idx % 10 == 0: print(f"Procesando juegos {idx}/{len(app_ids)}")
        
        try:
            r = requests.get(f"{base_url}?appids={app_id}&l=english")
            if r.status_code != 200: continue
            
            data = r.json()
            if not data or str(app_id) not in data or not data[str(app_id)]['success']: continue
                
            game_data = data[str(app_id)]['data']
            game_name = game_data.get('name', 'Unknown')
            pc_reqs = game_data.get('pc_requirements', {})
            
            # Procesar minimos
            if 'minimum' in pc_reqs:
                parsed = parse_requirements(clean_html_text(pc_reqs['minimum']))
                if parsed:
                    entry = {'app_id': app_id, 'game_name': game_name, **parsed}
                    min_data.append(entry)
            
            # Procesar recomendados
            if 'recommended' in pc_reqs:
                parsed = parse_requirements(clean_html_text(pc_reqs['recommended']))
                if parsed:
                    entry = {'app_id': app_id, 'game_name': game_name, **parsed}
                    rec_data.append(entry)
                
            # Pausa de cortesia para la API
            time.sleep(1.2)
            
        except Exception: continue

    return pd.DataFrame(min_data), pd.DataFrame(rec_data)

### 8. Ejecucion "main"

Finalmente, orquestamos todo el proceso. Al ejecutar esta celda:

1.  Se creara la estructura de carpetas.

2.  Se descargara y procesara la encuesta de hardware.

3.  Se generaran los rankings de componentes.

4.  Se obtendran los IDs de los juegos mas vendidos.

5.  Se extraeran y limpiaran sus requisitos.

6.  Todo se guardara en las carpetas `csv_data` y `json_data`.

In [8]:
def main():
    # Creando proceso de resolucion de datos
    create_structure()
    
    # PASO 1: Hardware
        # Analizando encuesta de hardware de Steam
    df_hw = get_steam_hardware_survey()
    save_datasets(df_hw, 'hw_survey_full')
    
        # Generando rankings de hardware
    save_ranked_hardware(df_hw)
    
    # PASO 2: IDs de Juegos
    print(f"\nObteniendo top {GAME_LIMIT} juegos mas vendidos...")
    ids = get_top_game_ids(limit=GAME_LIMIT)
    print(f"\nIDs recolectados: {len(ids)}\n")
    
    # PASO 3: Requisitos
        # Extrayendo y limpiando requisitos
    df_min, df_rec = get_game_requirements(ids)
    
    save_datasets(df_min, 'req_minimos')
    save_datasets(df_rec, 'req_recomendados')
    
    print("\n--- Proceso completado exitosamente ---")

if __name__ == "__main__":
    main()
# Autor: Munteanu Popa, Andrei


Obteniendo top 6000 juegos mas vendidos...
 Bloque 0-50 obtenido. Total: 50
 Bloque 50-100 obtenido. Total: 100
 Bloque 100-150 obtenido. Total: 150
 Bloque 150-200 obtenido. Total: 200
 Bloque 200-250 obtenido. Total: 250
 Bloque 250-300 obtenido. Total: 300
 Bloque 300-350 obtenido. Total: 350
 Bloque 350-400 obtenido. Total: 400
 Bloque 400-450 obtenido. Total: 450
 Bloque 450-500 obtenido. Total: 500
 Bloque 500-550 obtenido. Total: 550
 Bloque 550-600 obtenido. Total: 600
 Bloque 600-650 obtenido. Total: 650
 Bloque 650-700 obtenido. Total: 699
 Bloque 700-750 obtenido. Total: 749
 Bloque 750-800 obtenido. Total: 799
 Bloque 800-850 obtenido. Total: 849
 Bloque 850-900 obtenido. Total: 899
 Bloque 900-950 obtenido. Total: 949
 Bloque 950-1000 obtenido. Total: 999
 Bloque 1000-1050 obtenido. Total: 1049
 Bloque 1050-1100 obtenido. Total: 1099
 Bloque 1100-1150 obtenido. Total: 1149
 Bloque 1150-1200 obtenido. Total: 1199
 Bloque 1200-1250 obtenido. Total: 1249
 Bloque 1250-1300 ob